**Copyright 2021 The TF-Agents Authors.**

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Actor-Learner APIを使用したSAC minitaur

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/agents/tutorials/7_SAC_minitaur_tutorial"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a> </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/agents/tutorials/7_SAC_minitaur_tutorial.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a> </td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/agents/tutorials/7_SAC_minitaur_tutorial.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub でソースを表示</a></td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/agents/tutorials/7_SAC_minitaur_tutorial.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a>   </td>
</table>


## 前書き

ここでは、[Minitaur](https://github.com/bulletphysics/bullet3/blob/master/examples/pybullet/gym/pybullet_envs/bullet/minitaur.py) 環境で [Soft Actor Critic](https://arxiv.org/abs/1812.05905) エージェントをトレーニングする方法を紹介します。

[DQN Colab](https://github.com/tensorflow/agents/blob/master/docs/tutorials/1_dqn_tutorial.ipynb) にすでに精通されている場合は、馴染みやすいと思います。主な違いは次のとおりです。

- エージェントを DQN から SAC に変更します。
- Minitaur（CartPole よりもはるかに複雑な環境）環境でのトレーニング。Minitaur 環境は、四足歩行ロボットが前進するようにトレーニングすることを目的としています。
- 分散型強化学習のための TF-Agent Actor-Learner API を使用します。

API は、経験再生バッファーと変数コンテナ（パラメータサーバー）を使用した分散データ収集と、複数のデバイスにわたる分散トレーニングの両方をサポートしています。API は非常にシンプルでモジュール化されています。再生バッファーと可変コンテナには [Reverb](https://deepmind.com/research/open-source/Reverb)、GPU と TPU での分散トレーニングには [TF DistributionStrategy API](https://www.tensorflow.org/guide/distributed_training) が使用されます。

以下の依存関係をインストールしていない場合は、実行します。

In [ ]:
!sudo apt-get update
!sudo apt-get install -y xvfb ffmpeg
!pip install 'imageio==2.4.0'
!pip install matplotlib
!pip install tf-agents[reverb]
!pip install pybullet

## セットアップ

まず、必要なさまざまなツールをインポートします。

In [ ]:
import base64
import imageio
import IPython
import matplotlib.pyplot as plt
import os
import reverb
import tempfile
import PIL.Image

import tensorflow as tf

from tf_agents.agents.ddpg import critic_network
from tf_agents.agents.sac import sac_agent
from tf_agents.agents.sac import tanh_normal_projection_network
from tf_agents.environments import suite_pybullet
from tf_agents.metrics import py_metrics
from tf_agents.networks import actor_distribution_network
from tf_agents.policies import greedy_policy
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_py_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.train import actor
from tf_agents.train import learner
from tf_agents.train import triggers
from tf_agents.train.utils import spec_utils
from tf_agents.train.utils import strategy_utils
from tf_agents.train.utils import train_utils

tempdir = tempfile.gettempdir()

## ハイパーパラメータ

In [ ]:
env_name = "MinitaurBulletEnv-v0" # @param {type:"string"}

# Use "num_iterations = 1e6" for better results (2 hrs)
# 1e5 is just so this doesn't take too long (1 hr)
num_iterations = 100000 # @param {type:"integer"}

initial_collect_steps = 10000 # @param {type:"integer"}
collect_steps_per_iteration = 1 # @param {type:"integer"}
replay_buffer_capacity = 10000 # @param {type:"integer"}

batch_size = 256 # @param {type:"integer"}

critic_learning_rate = 3e-4 # @param {type:"number"}
actor_learning_rate = 3e-4 # @param {type:"number"}
alpha_learning_rate = 3e-4 # @param {type:"number"}
target_update_tau = 0.005 # @param {type:"number"}
target_update_period = 1 # @param {type:"number"}
gamma = 0.99 # @param {type:"number"}
reward_scale_factor = 1.0 # @param {type:"number"}

actor_fc_layer_params = (256, 256)
critic_joint_fc_layer_params = (256, 256)

log_interval = 5000 # @param {type:"integer"}

num_eval_episodes = 20 # @param {type:"integer"}
eval_interval = 10000 # @param {type:"integer"}

policy_save_interval = 5000 # @param {type:"integer"}

## 環境変数

RL の環境は、解決しようとしているタスクまたは問題を表しています。標準環境は、`suites`を使用して TF-Agent で簡単に作成できます。OpenAI Gym、Atari、DM Control などのソースから環境を読み込むためのさまざまな`suites`が用意されています。これには、文字列の環境名が与えられます。

では、PybulletスイートからMinituar環境を読み込みましょう。

In [ ]:
env = suite_pybullet.load(env_name)
env.reset()
PIL.Image.fromarray(env.render())

この環境での目標は、エージェントがMinitaurロボットを制御し、可能な限り速く前進させられるようにポリシーをトレーニングすることです。エピソードには1000ステップあり、リターンはエピソード全体の報酬の合計になります。

環境が`観察`として提供する情報を見てみましょう。ポリシーは観察を使用して`行動`を生成します。

In [ ]:
print('Observation Spec:')
print(env.time_step_spec().observation)
print('Action Spec:')
print(env.action_spec())

The observation is fairly complex. We receive 28 values representing the angles, velocities, and torques for all the motors. In return the environment expects 8 values for the actions between `[-1, 1]`. These are the desired motor angles.

通常、2つの環境を作成します。1つ目はトレーニング中にデータを収集するため、もう2つ目は評価のための環境です。環境は純粋なpythonで記述され、Actor Learner APIが直接使用するnumpy配列を使用します。

In [ ]:
collect_env = suite_pybullet.load(env_name)
eval_env = suite_pybullet.load(env_name)

## 分散ストラテジー

DistributionStrategy APIでは、データの並列処理を使用して、複数のGPUやTPUなどの複数のデバイス間でトレーニングステップの計算を実行できます。トレーニングステップは以下のとおりです。

- トレーニングデータのバッチを受けとる
- デバイス間で分割する
- 前進ステップを計算する
- 損失のMEANを集計して計算する
- 後退ステップを計算し、勾配変数の更新を実行する

TF-Agent Learner API と DistributionStrategy API を使用すると、以下のトレーニングロジックを変更せずに、トレーニングステップの実行を GPU（MirroredStrategy を使用）からTPU（TPUStrategyを使用）に簡単に切り替えることができます。

### GPUを有効にする

GPU で実行する場合は、まずノートブックで GPU を有効にする必要があります。

- ［編集］→［ノートブック設定］に移動します
- ［ハードウェアアクセラレータ］ドロップダウンから GPU を選択します

### ストラテジーの選択

`strategy_utils`を使用してストラテジーを生成します。 内部的に、パラメータを渡します。

- `use_gpu = False`は`tf.distribute.get_strategy()`を返します。 この場合、CPU を使用します。
- `use_gpu = True`は`tf.distribute.MirroredStrategy()`を返します。この場合、1 台のマシンのTensorFlowにより認識されるすべての GPU を使用します。

In [ ]:
use_gpu = True #@param {type:"boolean"}

strategy = strategy_utils.get_strategy(tpu=False, use_gpu=use_gpu)

以下に示すように、すべての変数とエージェントは`strategy.scope()`の下に作成する必要があります。

## エージェント

SAC エージェントを作成するには、まず、トレーニングするネットワークを作成する必要があります。SAC は actor-critic エージェントなので、2 つのネットワークを必要とします。

critic は、`Q(s,a)` の値を推定します。つまり、入力として観測と行動を受け取り、状態に対して行動がどのくらい効果的であるかを推定します。


In [ ]:
observation_spec, action_spec, time_step_spec = (
      spec_utils.get_tensor_specs(collect_env))

with strategy.scope():
  critic_net = critic_network.CriticNetwork(
        (observation_spec, action_spec),
        observation_fc_layer_params=None,
        action_fc_layer_params=None,
        joint_fc_layer_params=critic_joint_fc_layer_params,
        kernel_initializer='glorot_uniform',
        last_kernel_initializer='glorot_uniform')

このcriticを使用して、`actor`ネットワークをトレーニングします。これにより、与えられた観察に対する行動を生成できます。

`ActorNetwork`は、tanh-squashed [MultivariateNormalDiag](https://www.tensorflow.org/probability/api_docs/python/tfp/distributions/MultivariateNormalDiag) 分布のパラメータを予測します。行動を生成する必要がある場合は常に、その時点の観測を条件としてこの分布がサンプリングされます。

In [ ]:
with strategy.scope():
  actor_net = actor_distribution_network.ActorDistributionNetwork(
      observation_spec,
      action_spec,
      fc_layer_params=actor_fc_layer_params,
      continuous_projection_net=(
          tanh_normal_projection_network.TanhNormalProjectionNetwork))

これらのネットワークを使用して、エージェントをインスタンス化できます。


In [ ]:
with strategy.scope():
  train_step = train_utils.create_train_step()

  tf_agent = sac_agent.SacAgent(
        time_step_spec,
        action_spec,
        actor_network=actor_net,
        critic_network=critic_net,
        actor_optimizer=tf.keras.optimizers.Adam(
            learning_rate=actor_learning_rate),
        critic_optimizer=tf.keras.optimizers.Adam(
            learning_rate=critic_learning_rate),
        alpha_optimizer=tf.keras.optimizers.Adam(
            learning_rate=alpha_learning_rate),
        target_update_tau=target_update_tau,
        target_update_period=target_update_period,
        td_errors_loss_fn=tf.math.squared_difference,
        gamma=gamma,
        reward_scale_factor=reward_scale_factor,
        train_step_counter=train_step)

  tf_agent.initialize()

## 再生バッファ

環境から収集されたデータを追跡するためには [Reverb](https://deepmind.com/research/open-source/Reverb) を使用します。Reverb は、Deepmind により開発された効率的かつ拡張可能で使いやすい再生システムです。これは、Actor により収集され、トレーニング中に Learner により消費される経験データを格納します。

このチュートリアルでは、`max_size`よりも重要ではありませんが、非同期収集とトレーニングを使用する分散設定では、2〜1000 の samples_per_insert を使用して、`rate_limiters.SampleToInsertRatio`を試すことをお勧めします。以下に例を示します。

```
rate_limiter=reverb.rate_limiters.SampleToInsertRatio(samples_per_insert=3.0, min_size_to_sample=3, error_buffer=3.0)
```


In [ ]:
table_name = 'uniform_table'
table = reverb.Table(
    table_name,
    max_size=replay_buffer_capacity,
    sampler=reverb.selectors.Uniform(),
    remover=reverb.selectors.Fifo(),
    rate_limiter=reverb.rate_limiters.MinSize(1))

reverb_server = reverb.Server([table])

再生バッファは、格納されるテンソルを記述する仕様を使用して構築されます。これは、`tf_agent.collect_data_spec`を使用してエージェントから取得できます。

SAC エージェントは損失を計算するためにその時点と次の両方の観測を必要とするため、`sequence_length=2`を設定します。

In [ ]:
reverb_replay = reverb_replay_buffer.ReverbReplayBuffer(
    tf_agent.collect_data_spec,
    sequence_length=2,
    table_name=table_name,
    local_server=reverb_server)

次に、Reverb再生バッファーからTensorFlowデータセットを生成します。これをLearnerに渡して、トレーニングの体験をサンプリングします。

In [ ]:
dataset = reverb_replay.as_dataset(
      sample_batch_size=batch_size, num_steps=2).prefetch(50)
experience_dataset_fn = lambda: dataset

## ポリシー

TF-Agent では、ポリシーは RL のポリシーの標準的な概念を表します。`time_step`が指定されると、行動または行動の分布が生成されます。主なメソッドは`policy_step = policy.step(time_step)`で、`policy_step`は、名前付きのタプル`PolicyStep(action, state, info)`です。`policy_step.action`は、環境に適用される`action`で、`state`は、ステートフル（RNN）ポリシーの状態を表し、`info`には行動のログ確率などの補助情報が含まれる場合があります。

エージェントには 2 つのポリシーが含まれています。

- `agent.policy` — 評価と導入に使用される主なポリシー。
- `agent.collect_policy` — データ収集に使用される補助的なポリシー。

In [ ]:
tf_eval_policy = tf_agent.policy
eval_policy = py_tf_eager_policy.PyTFEagerPolicy(
  tf_eval_policy, use_tf_function=True)

In [ ]:
tf_collect_policy = tf_agent.collect_policy
collect_policy = py_tf_eager_policy.PyTFEagerPolicy(
  tf_collect_policy, use_tf_function=True)

ポリシーはエージェントとは無関係に作成できます。たとえば、`tf_agents.policies.random_tf_policy`を使用して、各<code>time_step</code>の行動をランダムに選択するポリシーを作成できます。

In [ ]:
random_policy = random_py_policy.RandomPyPolicy(
  collect_env.time_step_spec(), collect_env.action_spec())

## アクター

Actorは、ポリシーと環境の間の相互作用を管理します。

- Actorコンポーネントには、環境のインスタンス（`py_environment`として）とポリシー変数のコピーが含まれています。
- ポリシー変数のローカル値が指定されると、各Actorワーカーは、一連のデータ収集ステップを実行します。
- 変数の更新は、`actor.run()`を呼び出す前に、トレーニングスクリプトの変数コンテナクライアントインスタンスを使用して明示的に行われます。
- 観測された経験は、各データ収集ステップで再生バッファーに書き込まれます。

Actor がデータ収集ステップを実行すると、状態、行動、報酬）のTrajectoryをオブザーバーに渡し、オブザーバーはそれらをキャッシュして Reverb 再生システムに書き込みます。

`stride_length=1`なので、フレーム [(t0,t1) (t1,t2) (t2,t3), ...] の Trajectory を保存します。

In [ ]:
rb_observer = reverb_utils.ReverbAddTrajectoryObserver(
  reverb_replay.py_client,
  table_name,
  sequence_length=2,
  stride_length=1)

ランダムなポリシーで Actor を作成し、再生バッファーをシードする経験を収集します。

In [ ]:
initial_collect_actor = actor.Actor(
  collect_env,
  random_policy,
  train_step,
  steps_per_run=initial_collect_steps,
  observers=[rb_observer])
initial_collect_actor.run()

収集ポリシーを使用して Actor をインスタンス化し、トレーニング中にさらに経験を収集します。

In [ ]:
env_step_metric = py_metrics.EnvironmentSteps()
collect_actor = actor.Actor(
  collect_env,
  collect_policy,
  train_step,
  steps_per_run=1,
  metrics=actor.collect_metrics(10),
  summary_dir=os.path.join(tempdir, learner.TRAIN_DIR),
  observers=[rb_observer, env_step_metric])

トレーニング中にポリシーを評価するために使用される Actor を作成します。後でメトリックを記録するために`actor.eval_metrics(num_eval_episodes)`を渡します。

In [ ]:
eval_actor = actor.Actor(
  eval_env,
  eval_policy,
  train_step,
  episodes_per_run=num_eval_episodes,
  metrics=actor.eval_metrics(num_eval_episodes),
  summary_dir=os.path.join(tempdir, 'eval'),
)

## 学習者

Learner コンポーネントにはエージェントが含まれており、再生バッファーからの経験データを使用して、ポリシー変数への勾配ステップの更新を実行します。1 つ以上のトレーニングステップの後、Learner は新しい一連の変数値を変数コンテナにプッシュできます。

In [ ]:
saved_model_dir = os.path.join(tempdir, learner.POLICY_SAVED_MODEL_DIR)

# Triggers to save the agent's policy checkpoints.
learning_triggers = [
    triggers.PolicySavedModelTrigger(
        saved_model_dir,
        tf_agent,
        train_step,
        interval=policy_save_interval),
    triggers.StepPerSecondLogTrigger(train_step, interval=1000),
]

agent_learner = learner.Learner(
  tempdir,
  train_step,
  tf_agent,
  experience_dataset_fn,
  triggers=learning_triggers,
  strategy=strategy)

## 指標と評価

上記の`actor.eval_metrics`でevalアクターをインスタンス化しました。これにより、ポリシー評価中に最も一般的に使用されるメトリックが作成されます。

- 平均リターン。 リターンは、エピソードの環境でポリシーを実行しているときに得られる報酬の合計であり、通常、これをいくつかのエピソードで平均します。
- エピソードの長さの平均。

Actorを実行して、これらのメトリクスを生成します。

In [ ]:
def get_eval_metrics():
  eval_actor.run()
  results = {}
  for metric in eval_actor.metrics:
    results[metric.name] = metric.result()
  return results

metrics = get_eval_metrics()

In [ ]:
def log_eval_metrics(step, metrics):
  eval_results = (', ').join(
      '{} = {:.6f}'.format(name, result) for name, result in metrics.items())
  print('step = {0}: {1}'.format(step, eval_results))

log_eval_metrics(0, metrics)

さまざまなメトリクスの他の標準実装については、[metrics module](https://github.com/tensorflow/agents/blob/master/tf_agents/metrics/tf_metrics.py)をご覧ください。

## エージェントのトレーニング

トレーニングループには、環境からのデータ収集とエージェントのネットワークの最適化の両方が含まれます。途中で、エージェントのポリシーを時々評価して、状況を確認します。

In [ ]:
#@test {"skip": true}
try:
  %%time
except:
  pass

# Reset the train step
tf_agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = get_eval_metrics()["AverageReturn"]
returns = [avg_return]

for _ in range(num_iterations):
  # Training.
  collect_actor.run()
  loss_info = agent_learner.run(iterations=1)

  # Evaluating.
  step = agent_learner.train_step_numpy

  if eval_interval and step % eval_interval == 0:
    metrics = get_eval_metrics()
    log_eval_metrics(step, metrics)
    returns.append(metrics["AverageReturn"])

  if log_interval and step % log_interval == 0:
    print('step = {0}: loss = {1}'.format(step, loss_info.loss.numpy()))

rb_observer.close()
reverb_server.stop()

## 可視化


### プロット

エージェントのパフォーマンスを確認するために、平均利得とグローバルステップをプロットできます。`Minitaur`では、報酬関数は、Minitaur が 1000 ステップで歩く距離に基づいており、エネルギー消費にペナルティを課します。

In [ ]:
#@test {"skip": true}

steps = range(0, num_iterations + 1, eval_interval)
plt.plot(steps, returns)
plt.ylabel('Average Return')
plt.xlabel('Step')
plt.ylim()

### ビデオ

各ステップで環境をレンダリングすると、エージェントのパフォーマンスを可視化できます。その前に、この Colab に動画を埋め込む関数を作成しましょう。

In [ ]:
def embed_mp4(filename):
  """Embeds an mp4 file in the notebook."""
  video = open(filename,'rb').read()
  b64 = base64.b64encode(video)
  tag = '''
  <video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
  Your browser does not support the video tag.
  </video>'''.format(b64.decode())

  return IPython.display.HTML(tag)

次のコードは、いくつかのエピソードに渡るエージェントのポリシーを可視化します。

In [ ]:
num_episodes = 3
video_filename = 'sac_minitaur.mp4'
with imageio.get_writer(video_filename, fps=60) as video:
  for _ in range(num_episodes):
    time_step = eval_env.reset()
    video.append_data(eval_env.render())
    while not time_step.is_last():
      action_step = eval_actor.policy.action(time_step)
      time_step = eval_env.step(action_step.action)
      video.append_data(eval_env.render())

embed_mp4(video_filename)